sources:
https://colab.research.google.com/github/digitalepidemiologylab/covid-twitter-bert/blob/master/CT_BERT_Huggingface_(GPU_training).ipynb#scrollTo=11kI0lhIJoxd
https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671

In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

In [2]:
# model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading: 100%|██████████| 570/570 [00:00<00:00, 63.0kB/s]
Downloading: 100%|██████████| 511M/511M [00:31<00:00, 17.2MB/s] 
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Downloading: 100%|██████████| 226k/226k [00:01<00:00, 142kB/s]  
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 3.47kB/s]


In [3]:
# odel.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [2]:
model_name = 'digitalepidemiologylab/covid-twitter-bert' # CT-BERT

In [3]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForSequenceClassification,
   AdamW,
   glue_convert_examples_to_features
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [13]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

Downloading: 100%|██████████| 1.47G/1.47G [01:11<00:00, 20.5MB/s] 
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Small function only used for formatting the output
def format_prediction(preds, label_mapping, label_name):
    preds = tf.nn.softmax(preds, axis=1)
    formatted_preds = []
    for pred in preds.numpy():
        # convert to Python types and sort
        pred = {label: float(probability) for label, probability in zip(label_mapping.values(), pred)}
        pred = {k: v for k, v in sorted(pred.items(), key=lambda item: item[1], reverse=True)}
        formatted_preds.append({label_name: list(pred.keys())[0], f'{label_name}_probabilities': pred})
    return formatted_preds

In [7]:
import tensorflow as tf
import json
import pandas as pd
import tensorflow_datasets as tfds

In [8]:
data = pd.read_excel('tweets_semi_labeled.xlsx')

In [9]:
tweets = data['full_text'][:200]
labels = data['Label (critical: 1/neutral: 0)'][:200]

In [10]:
df = pd.DataFrame({'tweets': tweets, 'labels': labels})
# print(good_df.head())

In [11]:
label_mapping = {'critical': 1, 'neutral': 0}
# train_size = 150


In [12]:

tf_data = (
    tf.data.Dataset.from_tensor_slices(
        (tf.cast(df['tweets'].values, tf.string), tf.cast(df['labels'].values, tf.int64))
))

In [11]:
# label_mapping = {"0": "negative", "1": "positive"} # only for testing purposes

In [27]:
input_text = 'i love you'
# Tokenize the input 
input_ids = tf.constant(tokenizer.encode(input_text, add_special_tokens=True))[None, :]
# Run predictions
preds = model(input_ids)
# format logits
formatted_preds = format_prediction(preds[0], label_mapping, 'sentiment')

print(f'\nLabel Mapping:{json.dumps(label_mapping, indent=4)}')
print(f'\nLogits: {preds}')
print(f'\nProbabilities:{json.dumps(formatted_preds, indent=4)}')


Label Mapping:{
    "critical": 1,
    "neutral": 0
}

Logits: TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.17656091, -0.30736065]], dtype=float32)>, hidden_states=None, attentions=None)

Probabilities:[
    {
        "sentiment": 1,
        "sentiment_probabilities": {
            "1": 0.6186734437942505,
            "0": 0.3813265264034271
        }
    }
]
